### Sudachi

In [1]:
from sudachipy import tokenizer
from sudachipy import dictionary

tokenizer_obj = dictionary.Dictionary().create()

In [2]:
mode = tokenizer.Tokenizer.SplitMode.A
[m.surface() for m in tokenizer_obj.tokenize("国家公務員", mode)]

['国家', '公務', '員']

In [3]:
mode = tokenizer.Tokenizer.SplitMode.B
[m.surface() for m in tokenizer_obj.tokenize("国家公務員", mode)]

['国家', '公務員']

In [4]:
mode = tokenizer.Tokenizer.SplitMode.C
[m.surface() for m in tokenizer_obj.tokenize("国家公務員", mode)]

['国家公務員']

### Mecab

In [5]:
import MeCab

# Neologdの読み込み
tagger = MeCab.Tagger('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')

input = 'あいみょんときゃりーぱみゅぱみゅ。'

result = tagger.parse(input)
print(result)

node = tagger.parseToNode(input)
target_parts_of_speech = ('名詞', )
while node:
    if node.feature.split(',')[0] in target_parts_of_speech:
        print(node.surface)
    node = node.next

あいみょん	名詞,固有名詞,人名,一般,*,*,あいみょん,アイミョン,アイミョン
と	助詞,並立助詞,*,*,*,*,と,ト,ト
きゃりーぱみゅぱみゅ	名詞,固有名詞,人名,一般,*,*,きゃりーぱみゅぱみゅ,キャリーパミュパミュ,キャリーパミュパミュ
。	記号,句点,*,*,*,*,。,。,。
EOS

あいみょん
きゃりーぱみゅぱみゅ


### BERT(日本語)

In [6]:
# 参考:https://yukoishizaki.hatenablog.com/entry/2019/11/27/151018
import os
import torch
from transformers import BertForMaskedLM, BertConfig, BertTokenizer
from pyknp import Juman

BASE_PATH = '/home/jovyan/Japanese_L-12_H-768_A-12_E-30_BPE_WWM_transformers'
BERT_CONFIG = 'config.json'
BERT_MODEL = 'pytorch_model.bin'
VOCAVULARY_LIST = 'vocab.txt'

jumanpp = Juman()

# 形態素解析
text = 'どんなに勉強しても全然頭が良くならない'
result = jumanpp.analysis(text)
tokenized_text =[mrph.midasi for mrph in result.mrph_list()]
print(tokenized_text)

['どんなに', '勉強', 'して', 'も', '全然', '頭', 'が', '良く', 'なら', 'ない']


In [7]:
# Mask 
tokenized_text.insert(0, '[CLS]')
tokenized_text.append('[SEP]')

masked_index = 6 # Maskしたいtextのindex 
tokenized_text[masked_index] = '[MASK]'
print(tokenized_text)

['[CLS]', 'どんなに', '勉強', 'して', 'も', '全然', '[MASK]', 'が', '良く', 'なら', 'ない', '[SEP]']


In [8]:
# Bert model
config = BertConfig.from_json_file(os.path.join(BASE_PATH, BERT_CONFIG))
model = BertForMaskedLM.from_pretrained(os.path.join(BASE_PATH, BERT_MODEL), config=config)
tokenizer = BertTokenizer(os.path.join(BASE_PATH, VOCAVULARY_LIST), do_lower_case=False, do_basic_tokenize=False)

# token化
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
tokens_tensor = torch.tensor([indexed_tokens])
print(tokens_tensor)

Some weights of the model checkpoint at /home/jovyan/Japanese_L-12_H-768_A-12_E-30_BPE_WWM_transformers/pytorch_model.bin were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[   2,    1, 6547,   19,   23,    1,    4,   11, 4161,  371,   46,    3]])


In [9]:
# 予測
model.eval()

tokens_tensor = tokens_tensor.to('cuda')
model.to('cuda')

with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]

_, predicted_indexes = torch.topk(predictions[0, masked_index], k=5)
predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_indexes.tolist())
print(predicted_tokens)

['成績', '頭', '気持ち', '方', '態度']
